In [637]:
import requests
import time
from bs4 import BeautifulSoup
import regex as re


In [638]:
url_donutshops = "https://www.yelp.com/search?find_desc=donut+shop&find_loc=San+Francisco%2C+CA&start="

header = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36" ,
"authority": "www.tagesschau.de",
"method": "GET",
"path":"/",
"scheme":"https",
"accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
"accept-encoding": "gzip, deflate, br",
"accept-language": "en-US,en;q=0.9,de;q=0.8",
"cache-control": "max-age=0",
"cookie": "atuserid=%7B%22name%22%3A%22atuserid%22%2C%22val%22%3A%2257ea5dd6-4c35-4982-942f-8a7f8b8c3a4b%22%2C%22options%22%3A%7B%22end%22%3A%222023-02-17T05%3A02%3A59.936Z%22%2C%22path%22%3A%22%2F%22%7D%7D; atidvisitor=%7B%22name%22%3A%22atidvisitor%22%2C%22val%22%3A%7B%22vrn%22%3A%22-595936-%22%7D%2C%22options%22%3A%7B%22path%22%3A%22%2F%22%2C%22session%22%3A15724800%2C%22end%22%3A15724800%7D%7D",
"sec-ch-ua-mobile": "?0",
"sec-ch-ua-platform": "Windows",
"sec-fetch-dest": "document",
"sec-fetch-mode": "navigate",
"sec-fetch-site": "none",
"sec-fetch-user": "?1",
"upgrade-insecure-requests": "1"}

In [640]:
############## Part 1 - getting the Yelp pages ##################

for i in range(0,40,10):
    
    #Setting a page number for each page
    page_num = int((i/10)+1)
    
    page_url = f"{url_donutshops}{str(i)}"
    
    #Getting the page into a variable
    yelp_page = requests.get(url = page_url, headers = header)

    #Creating a new file
    yelp_html = open(f"sf_donut_shop_search_page_{str(page_num)}.html", "w")
    #The parameter "w" creates a file only if it does not exist already


    #Adding the html content to the page using the write() function

    yelp_html.write(yelp_page.text)
    
    time.sleep(5)

print("The Yelp search pages have been stored. \n")
print("\n ################## END OF PART 1 ##################")

The Yelp search pages have been stored. 


 ################## END OF PART 1 ##################


In [642]:
######### Part 2 - getting shop data ##################

#Storing all the values in separate blank lists 

rest_ranks = ['']*40
rest_name = ['']*40
rest_url = ['']*40
rest_rating = ['']*40
rest_reviews = ['']*40
rest_store_tags = ['']*40
rest_price_range = ['']*40
rest_delivery_tags = ['']*40
rest_order_on_yelp_flag = ['']*40

for page_num in range(1,5):
    
    yelp_read = open(f"sf_donut_shop_search_page_{str(page_num)}.html", "r")
    yelp_parsed = BeautifulSoup(yelp_read, 'lxml')

    #Finding all the unordered lists of class "srp-results srp-list clearfix"
    full_list = yelp_parsed.find("ul",class_="undefined list__09f24__ynIEd")
    
    yelp_url = "https://www.yelp.com/"
    all_list_items = full_list.find_all("li")


    for j in range(0,len(all_list_items)):
        
        try:
            
            #Non-sponsored places start from the 11th element on the list

            this_restaurant = all_list_items[j]
            restaurant_span = this_restaurant.find_all("span", {'class' : 'css-1uq0cfn'})[0]
            rank_name = restaurant_span.text
            
            #Checking if the result is non-sponsored
            try:
                rest_rank = int(int(re.split('\.\s', rank_name)[0])/1) #This will filter out sponsored results
                
                rest_name[rest_rank-1] = re.split('\.\s', rank_name)[1]

                #Extracting the name from the text
                print(f"\nName: {rest_name[rest_rank-1]}")

                #Extracting the rank from the text
                rest_ranks[rest_rank-1] = rest_rank
                
                print(f"Rank: {rest_ranks[rest_rank-1]}")
                
                #Getting the URL
                rest_url[rest_rank-1] = f"{yelp_url}{restaurant_span.find_all('a')[0].get('href')}"
                print(f"URL: {rest_url[rest_rank-1]}")

                #####Getting the star ratings

                #The star ratings class starts with i-stars
                star_rating_regex = re.compile('i-stars.*')

                #Getting the div into a variable
                star_rating_div = this_restaurant.find_all("div", {'class' : star_rating_regex})[0]
                rest_rating[rest_rank-1] = re.split('\sstar\srating',star_rating_div.get("aria-label"))[0]
                print(f"Star rating: {rest_rating[rest_rank-1]}")

                #####Getting number of reviews
                review_count_regex = re.compile('reviewCount.*')
                review_count_span = this_restaurant.find_all("span",{'class' : review_count_regex})
                rest_reviews[rest_rank-1] = review_count_span[0].text
                print(f"Reviews: {rest_reviews[rest_rank-1]}")

                #####Getting store tags
                store_tag_regex = re.compile('.*display--inline.*border-color.*')
                store_tag_list = this_restaurant.find_all("p", {'class' : 'css-1p8aobs'})

                #Creating an empty list to add categories of store
                category_list = list(range(0,len(store_tag_list)))

                #Storing the store tags in a list
                for category in range(0,len(store_tag_list)):
                    category_list[category] = store_tag_list[category].text

                rest_store_tags[rest_rank-1] = ", ".join(category_list)
                print(f"Store Tags: {rest_store_tags[rest_rank-1]}")

                #####Getting price range
                price_range_regex = re.compile('priceRange.*')
                rest_price_range[rest_rank-1] = this_restaurant.find_all("span", {'class' : price_range_regex})[0].text
                print(f"Price range: {rest_price_range[rest_rank-1]}")

                #####Getting delivery/dine-in tags
                try:
                    delivery_tag_regex = re.compile('.*TRUSTED_PROPERTY.*')
                    delivery_tag_list = this_restaurant.find_all("div", {'data-testid': delivery_tag_regex})
                    delivery_tag_divs = delivery_tag_list[0].find_all("div")

                    check_cross = list(range(0,len(delivery_tag_divs)))
                    for delivery in range(0,len(delivery_tag_divs),2):

                        check_mark_regex = re.compile('.*icon.*checkmark.*')
                        cross_mark_regex = re.compile('.*icon.*close.*')

                        if len(delivery_tag_divs[delivery].find_all("span",{'class' : check_mark_regex})) == 0:
                            check_cross[delivery] = 'No'

                        else:
                            check_cross[delivery] = 'Yes'

                        check_cross[delivery+1] = delivery_tag_divs[delivery].find_all("span",{'class' : tag_text_regex})[0].text

                    rest_delivery_tags[rest_rank-1] = " ".join(check_cross)
                    print(f"Delivery tags: {rest_delivery_tags[rest_rank-1]}")

                except:
                    pass

                #####Getting whether we can order on Yelp
                order_on_yelp_regex = re.compile('.*css-1enow5j.*')
                order_on_yelp_list = this_restaurant.find_all("span",{'class': order_on_yelp_regex})
                if len(order_on_yelp_list) == 0:
                    rest_order_on_yelp_flag[rest_rank-1] = 'No'
                else:
                    rest_order_on_yelp_flag[rest_rank-1] = 'Yes'
                print(f"Order on Yelp: {rest_order_on_yelp_flag[rest_rank-1]}")
            except:
                pass
            
        except:
            pass
print("\n################## END OF PART 2 ##################")


Name: Bob’s Donuts & Pastry Shop
Rank: 1
URL: https://www.yelp.com//biz/bobs-donuts-and-pastry-shop-san-francisco?osq=donut+shop
Star rating: 4.5
Reviews: 3328
Store Tags: Bakeries, Donuts
Price range: $
Order on Yelp: No

Name: Uncle Benny’s Donut and Bagel
Rank: 2
URL: https://www.yelp.com//biz/uncle-bennys-donut-and-bagel-san-francisco?osq=donut+shop
Star rating: 4.5
Reviews: 389
Store Tags: Donuts, Bagels, Coffee & Tea
Price range: $$
Delivery tags: No Outdoor dining Yes Delivery Yes Takeout
Order on Yelp: Yes

Name: Twisted Donuts and Coffee
Rank: 3
URL: https://www.yelp.com//biz/twisted-donuts-and-coffee-san-francisco?osq=donut+shop
Star rating: 4.5
Reviews: 508
Store Tags: Donuts, Bakeries
Price range: $
Delivery tags: No Delivery Yes Takeout
Order on Yelp: No

Name: All Star Donuts
Rank: 4
URL: https://www.yelp.com//biz/all-star-donuts-san-francisco-3?osq=donut+shop
Star rating: 4
Reviews: 280
Store Tags: Donuts, Sandwiches, Coffee & Tea
Price range: $
Delivery tags: Yes Deliv

In [644]:
######### Part 3 - exporting to MongoDB ##################

import json
import pymongo
from pandas.io.json import json_normalize 

#Creating a dictionary and converting it to JSON with data of all the shops

donut_dict = {'shops': [{'rank': a,
               'name': b,
               'url' : c,
               'rating' : d,
               'reviews' : e,
               'store_tags' : f,
               'price_range' : g,
               'delivery_tags' : h,
               'order_on_yelp' : i
              } for a,b,c,d,e,f,g,h,i in 
              zip(
                  rest_ranks,
                  rest_name,
                  rest_url,
                  rest_rating,
                  rest_reviews,
                  rest_store_tags,
                  rest_price_range,
                  rest_delivery_tags,
                  rest_order_on_yelp_flag
              )]}

donut_json = json.dumps(donut_dict, indent=4)

############ Inserting into MongoDB server ############

client = pymongo.MongoClient("mongodb://localhost:27017/?readPreference=primary&appname=MongoDB%20Compass&directConnection=true&ssl=false")
mydb = client["donut_shops"] #Creating a database if it does not exist already
mycol = mydb["sf_donut_shops"] #Choosing the collection where data would be input

#Inserting into the collection on MongoDB
x = mycol.insert_many(donut_dict['shops'])

print("Results exported to MongoDB.")

print("\n################## END OF PART 3 ##################")


Results exported to MongoDB.

################## END OF PART 3 ##################


In [645]:
######### Part 4 - Downloading store pages from MongoDB URL ##################

url_from_mongo = list(mycol.find({}, {"_id":0, "url":1}))

urls = list(range(0,len(url_from_mongo)))

#Storing the URLs into the urls variable
for i in range(0,len(urls)):
    
    #Extracting the URL and storing into the list
    urls[i] = list(url_from_mongo[i].values())[0]
    
    #Accessing the URL and storing into a html file
    
    donut_shop = requests.get(url = urls[i], headers = header)

    #Creating a new file
    donut_shop_html = open(f"sf_donut_shop_{str(i+1)}.html", "w")
    #The parameter "w" creates a file only if it does not exist already

    
    #Adding the html content to the page using the write() function
    donut_shop_html.write(donut_shop.text)
    
    time.sleep(5)
    

print("The shop pages have been extracted.")
print("\n################## END OF PART 4 ##################")


The shop pages have been extracted.

################## END OF PART 4 ##################


In [646]:
######### Part 5 & 6a - Parsing shop details ##################


#Setting the Location API
url_position_api = "http://api.positionstack.com/v1/forward"

access_key = "33be073505d8a38c80a2684ab0c275c8"

#position_headers = {'access_key' : access_key, 'token ' + token}

rest_address = ['']*40
rest_phone = ['']*40
rest_website = ['']*40
rest_position = ['']*40

for i in range(0,40):
    
    shop_read = open(f"sf_donut_shop_{str(i+1)}.html", "r")
    shop_parsed = BeautifulSoup(shop_read, 'lxml')
    
    try:
        
        print(f"\n{rest_name[i]}")
        
        #Getting the address
        rest_address[i] = shop_parsed.find_all("p", class_ = "css-qyp8bo")[0].text
        print(f"Address: {rest_address[i]}")
        
        #Getting the phone number
        #Using RegEx to extract phone number patterns
        
        match_phone_number = re.search(r'\(\d{3}\)\s\d{3}-\d{4}', shop_parsed.text)
        
        # If-statement after search() tests if it succeeded
        if match_phone_number:
            rest_phone[i] = match_phone_number.group()
            print('Phone:', rest_phone[i]) #Prints if there is a phone number
            
        else:
            pass
        
        #Getting the website
        try:
            rest_website[i] = shop_parsed.find_all("a", 
                                                   {'class':'css-1um3nx',
                                                    'target': '_blank',
                                                    'rel': 'noopener',
                                                    'role': 'link'
                                                   })[0].text
            print('Website:',rest_website[i])
        except:
            pass
        
        #Getting the Geolocation
        
        position_param = {'access_key' : access_key, 'query' : rest_address[i]}
        
        rest_position_text = requests.get(url = url_position_api, params = position_param).text
        rest_position_json = json.loads(rest_position_text)
        
        latitude = str(rest_position_json['data'][0]['latitude'])
        longitude = str(rest_position_json['data'][0]['longitude'])
        
        rest_position[i] = latitude + ', ' + longitude
        print('Geolocation:',rest_position[i])
            

    except:
        pass

print("\nNew fields have been parsed - including geolocation")
print("\n################## END OF PART 5 & 6a ##################")


Bob’s Donuts & Pastry Shop
Address: 1621 Polk St San Francisco, CA 94109
Phone: (415) 776-3141
Website: http://www.bobsdonutssf.com
Geolocation: 37.791852, -122.421276

Uncle Benny’s Donut and Bagel
Address: 2049 Irving St San Francisco, CA 94122
Phone: (628) 219-0956
Geolocation: 37.763299, -122.480192

Twisted Donuts and Coffee
Address: 1243 Noriega St San Francisco, CA 94122
Phone: (415) 463-5437
Website: http://twisteddonutsf.com
Geolocation: 37.754095, -122.477233

All Star Donuts
Address: 901 Clement St San Francisco, CA 94118
Phone: (415) 221-9838
Geolocation: 37.782559, -122.468983

Bob’s Donuts
Address: 601 Baker St San Francisco, CA 94117
Phone: (415) 776-3141
Website: https://www.bobsdonutssf.com
Geolocation: 37.776539, -122.441854

Happy Donuts
Address: 3801 24th St San Francisco, CA 94114
Phone: (415) 285-5890
Geolocation: 37.751594, -122.427598

Mochill Mochi Donut
Address: 1737 Post St Ste 395 San Francisco, CA 94115
Phone: (650) 619-2462
Website: https://www.mochillsf.

In [647]:
######### Part 6b - Updating MongoDB and indexing rank ##################

#donut_dict['shops']

#Creating a dictionary and converting it to JSON with data of all the shops

donut_new_fields_dict = {'new_fields': [{'address': a,
               'phone': b,
               'website' : c,
               'geolocation' : d
              } for a,b,c,d in 
              zip(
                  rest_address,
                  rest_phone,
                  rest_website,
                  rest_position
              )]}

donut_new_fields_json = json.dumps(donut_new_fields_dict['new_fields'], indent=4)

new_fields = donut_new_fields_dict['new_fields']

for r in range(0,40):
    index_column = {"rank": r+1}
    new_columns = {"$set": new_fields[r]}
    add_new_columns = mycol.update_many(index_column, new_columns)

print("\n################## END OF PART 6b ##################")



################## END OF PART 6b ##################


In [649]:
########### Part 6c - Placing an index on the Rank column ##########

mycol.create_index("rank") #Placing index on the rank column

print(mycol.index_information())
print("\n################## END OF PART 6c ##################")

{'_id_': {'v': 2, 'key': [('_id', 1)]}, 'rank_1': {'v': 2, 'key': [('rank', 1)]}}

################## END OF PART 6c ##################
